In [1]:
%matplotlib inline
import plot
import market
import metrics
from decimal import Decimal
import withdrawal
import harvesting
import simulate
import lens
import pandas
import seaborn
from matplotlib import pyplot as plt
import matplotlib

In [2]:
seaborn.set(style='whitegrid')
seaborn.set_context('poster')

In [3]:
m = market.Returns_US_1871()

s = simulate.calc_lens(harvesting.N_60_RebalanceHarvesting,
          lambda p, h: withdrawal.ConstantDollar(p, h, rate=Decimal('.043')),
          30,
          lens.calc_success
         )
len(s[s == False])

5

In [29]:
def final_pv(annual):
    df = pandas.DataFrame(annual)
    return df.tail(1)['portfolio_post'].item().value_r

s = simulate.calc_lens(harvesting.N_60_RebalanceHarvesting,
                       withdrawal.ConstantDollar,
                       30,
                      lens.calc_years_sustained)
s.sort_values(ascending=True)[10:]

1909    8.954911903056932942321784882
1973    9.117623795629515542038027215
1905     9.76991963076840628052969574
1967    9.776578586727366352903639238
1899    10.34886193913264682936095300
                    ...              
1873    115.9564355488806293466222481
1982    117.2036377933489749105713971
1877    119.2861495191276010147107481
1871    123.7549285295987227737204574
1872    124.6745827494345535972726104
Length: 108, dtype: object

In [4]:
%%time
import multiprocess as mp

def mp_lens(x):
    harvest, withdraw, years, lens = x
    import simulate
    s = simulate.calc_lens(harvest, withdraw, years, lens)
    return s

with mp.Pool(processes=mp.cpu_count()) as pool:
    def make_args():
        for years in range(1, 31):
            for rate in range(20, 71):
                yield (lambda p: harvesting.AnnualRebalancing(p, Decimal('.6')),
                      lambda p, h: withdrawal.CostantDollar(p, h, rate=Decimal(rate) / 10 / 100),
                      years,
                      lens.calc_success)
    results = pool.map(mp_lens, make_args())
results

ModuleNotFoundError: No module named 'multiprocess'

In [5]:
%%time
results = []

for years in range(1, 31):
    for rate in range(20, 71):
        s = simulate.calc_lens(lambda p: harvesting.AnnualRebalancing(p, Decimal('.6'),
                              lambda p, h: withdrawal.ConstantDollar(p, h, rate=Decimal(rate) / 10 / 100),
                               years,
                               lens.calc_success
                              )
        f = len(s[s == False]) / len(s)
        results.append((years, rate, f))
    print(years)

SyntaxError: invalid syntax (<unknown>, line 10)

In [ ]:
df = pandas.DataFrame(data=results, columns=['Years', 'WD', 'Failure'])
df['WD'] = df['WD'] / 10
df['Failure'] = df['Failure'] * 100
df.head()

In [ ]:
df[df['Years'] == 20].plot(x='WD', y='Failure', title='20 years left')

In [ ]:
r = []
for i in range(1,31):
    df_i = df[df['Years'] == i]
    df_zero = df_i[df_i['Failure'] == 0]
    r.append((i, df_zero['WD'].max()))
m = pandas.DataFrame(data=r, columns=['Years Left', 'Safe Withdrawal Rate'])

In [ ]:
m.plot(x='Years Left', y='Safe Withdrawal Rate')